In [1]:
import pandas as pd
import datetime
import numpy as np
import joblib as jl
import math
import json as js
import os
import sys
import warnings
warnings.filterwarnings('ignore')



""" Determines how many steps or predictions to be made based on the given date """

def determine_steps(date, yu):
    
    diff = pd.to_datetime(date[0:13]+':00') - yu # THE VARIABLE yu REFERS TO THE DATETIME OF THE LAST RECORDED 
                                                           # DATA POINT IN THE DATA ON WHICH THE MODEL WAS FITTED
    steps = diff.days * 24 + diff.seconds // 3600
    return steps





""" Forecasts the weather conditions for the required number of 'steps' based on the date queried """

def forecst(date = 0):
    
    model = jl.load('my_predictor.sav')   # LOADS THE EXISTING MODEL - TRAINED IN THE TRAINER NOTEBOOK
    
    
    df_diff = pd.read_csv('double_differenced.csv') # LOADS THE DOUBLE DIFFERENCED DATASET - SAVED INTO THE CURRENT DIRECTORY
                                                    # BY THE TRAINER NOTEBOOK
    
    
    df_diff['datetime'] = [pd.to_datetime(df_diff['datetime'][i]) for i in range(len(df_diff))]
    df_diff = df_diff.set_index('datetime')
    
    ref_date = df_diff.index[-1]
    
    if flag == 1:
        steps = n_steps
    else:
        steps = determine_steps(date, ref_date)
    
    forecast_input_req = df_diff.values[-24:]
    fc_req = model.forecast(y=forecast_input_req, steps=steps)
    
    df_forecast_req = pd.DataFrame(fc_req, columns=df_diff.columns)
    
    df_forecast_req['datetime'] = pd.date_range(df_diff.index[-1], periods=steps+1, freq='H')[1:]
    df_forecast_req = df_forecast_req.set_index('datetime')
    
    return df_forecast_req,df_diff,steps





""" Function to restore values from the double differenced predicted output - presents the output in json format """

def resultant(x, how = 'precise'):
    
    revert = pd.read_csv('final_two.csv')      # revert REFERS TO THE FIRST TWO DATAPOINTS OF THE FINAL DATASET
                                               # THESE DATAPOINTS ARE NECESSARY TO RESTORE THE DOUBLE DIFFERENCED DATASET TO ORIGINAL VALUES
    revert['datetime'] = [pd.to_datetime(revert['datetime'][i]) for i in range(len(revert))]

    revert_i = revert.set_index('datetime').diff().dropna()
    revert_i = revert_i.append(x[1]).append(x[0])
    revert_i = revert_i.cumsum()

    revert_ii = revert.set_index('datetime').iloc[0:1]
    revert_ii = revert_ii.append(revert_i)
    revert_ii = revert_ii.cumsum()
    
    op = revert_ii.iloc[-x[2]:]
    op = op.reset_index(drop = False)
    op.columns = ['datetime','src2_temp','src2_pres','src2_hum','iot_temp','iot_pres','iot_hum']
    op['datetime'] = [str(op['datetime'][i]) for i in range(len(op))]
    
    if how == 'until':
        if output_mode == 'df':
            return op
        else:
            json = op.to_json()
            return json
    
    elif how == 'precise':
        op = op.iloc[-1:]
        if output_mode == 'df':
            return op
        else:
            return js.dumps(op.to_dict('records')[0])

    

    

if __name__ == "__main__":
    
    flag = 0
    
    input_mode = input('Which option best describes your input? "no.ofDays", "no.ofSteps", "datestring", "jsonfile", or "jsonstring": ')
    if input_mode not in ['no.ofDays','no.ofSteps','datestring','jsonfile','jsonstring']:
        print('\n','enter a valid mode : "no.ofDays" or "no.ofSteps" or "datestring" or "jsonfile" or "jsonstring"')
        sys.exit()
    
    output_mode = input('How do you want your output? "json" or "df": ')
    if output_mode not in ['json','df']:
        print('\n','enter a valid mode : "json" or "df"')
        sys.exit()
    
    if input_mode != 'no.ofDays':
        how = input('Enter the method of forecast - "precise" or "until": ')
        if how not in ['precise','until']:
            print('\n','enter valid method')
            sys.exit()
    
    if input_mode == 'no.ofSteps':
        n_steps = input('Enter the number of steps to be predicted. It should be a whole number: ')
        try:
            n_steps = int(n_steps)
        except:
            sys.exit()
        flag = 1
        answer = resultant(forecst(),how= how)
        print('\n','\n',answer)
        
    
    
    elif input_mode == 'jsonfile':
        path = input('Enter path to your folder where the input json formatted data is present: ')
        valid = (".json")
        for f in os.listdir(path):
            if f.lower().endswith(valid):
                with open(os.path.join(path,f)) as free:
                    data = js.load(free)
                    date = data['datetime']
                    answer = resultant(forecst(date),how= how)
                    print('\n','\n',answer)
                    
    elif input_mode == 'datestring':
        date = input('Enter the datetime you want a prediction for: ')
        answer = resultant(forecst(date), how = how)
        print('\n','\n',answer)
        
    
    elif input_mode == 'no.ofDays':
        day = input('Enter number of days into the future you want a prediction for: ')
        date = str(datetime.datetime.now()+ datetime.timedelta(days = int(day)))
        answer = resultant(forecst(date),how = 'until')
        print('\n','\n',answer)
    
    else:
        input_str = input('Enter your JSON input: ')
        a_json = js.loads(input_str)
        date = a_json['datetime']
        answer = resultant(forecst(date), how = how)
        print('\n','\n',answer)

Which option best describes your input? "no.ofDays", "no.ofSteps", "datestring", "jsonfile", or "jsonstring": datestring
How do you want your output? "json" or "df": json
Enter the method of forecast - "precise" or "until": precise
Enter the datetime you want a prediction for: 2020-01-04T06:00:00Z

 
 {"datetime": "2020-01-04 06:00:00", "src2_temp": 38.06974375492453, "src2_pres": 1017.565660793963, "src2_hum": 65.06393092770718, "iot_temp": 47.18217261158953, "iot_pres": 98247.27592177768, "iot_hum": 56.2905820575574}


In [2]:
# answer
final_submission = answer[['datetime','iot_temp','iot_pres','iot_hum']]
final_submission.to_csv('final_submission.csv', index = False)